In [2]:
import tests.custom_tests as tests

N, values, weights, W_capacity = tests.easy_test()

In [3]:
import dimod
import dimod.utilities
from ncon import ncon


# https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/generated/dimod.binary.BinaryQuadraticModel.to_ising.html

x = [dimod.Binary(i) for i in range(N)]

cost = - dimod.quicksum(values[i][j] * x[i] * x[j] for i in range(N) for j in range (i+1)) # values
cost -= 0.9603*(W_capacity - dimod.quicksum(weights[i] * x[i] for i in range(N)))
cost += 0.0371*(W_capacity - dimod.quicksum(weights[i] * x[i] for i in range(N)))**2

model = dimod.BinaryQuadraticModel(cost.linear, cost.quadratic, cost.offset, vartype='BINARY')
H = model.to_ising()

print(H)

({1: -63.986999999999995, 0: -40.618500000000004, 2: -69.6055}, {(0, 1): -3.79, (2, 1): 9.88, (2, 0): 5.065}, -140.631)


In [4]:
import numpy as np
from DMRG.annealing_ham_to_MPO import from_ham_coeff

h_coeffs = [H[0][site] for site in range (N)] # H[0] contains linear interactions

J_coeffs = np.zeros((N,N))
for term in H[1]: # quadratic interacions
    # TODO in formulation i>j, but in MPO i<j (triangular superior, la meitat buida)
    J_coeffs[min(term[0], term[1]), max(term[0], term[1])] = H[1][term]

offset = H[2]
s = 1

M = from_ham_coeff(N, J_coeffs, h_coeffs, s)

print(len(M))

3


In [5]:
# Check M by doing diagonalization

M_copy = M.copy()
M_copy[0] = M_copy[0][0, :]
M_copy[-1] = M_copy[-1][:, -1]

'''for i in range(len(M)):
    print(M_copy[i].shape)'''


H = ncon(M_copy, [[1, -1, -2], [1, 2, -4, -5], [2,-6,-7]])

# indices_con = [[i, i+1, -i*2, -i*2-1] for i in range(1,len(M_copy))]
H = H.reshape(8,8)
print(H)

import scipy.linalg as la

D = la.eigvalsh(H)
print(D)
EnExact = 2 * sum(D[D < 0])
EnExact

'''SURT MALAMENT PERQUE ELS COEFF DE H NO ESTAN ON TOCA'''

[[-163.056    0.       0.     -53.735    0.       0.       0.       0.   ]
 [   0.       0.       0.       0.     -47.262    0.       0.     101.579]
 [   0.       0.       0.       0.       0.       0.       0.       0.   ]
 [   0.       0.       0.       0.       0.       0.       0.       0.   ]
 [   0.       0.       0.       0.       0.       0.       0.       0.   ]
 [   0.       0.       0.       0.       0.       0.       0.       0.   ]
 [ -84.369    0.       0.      45.212    0.       0.       0.       0.   ]
 [   0.       0.       0.       0.      16.265    0.       0.     185.366]]
[-200.4666963   -26.1560435    -1.41635555    0.            0.
    0.           63.5667398   186.78235555]


'SURT MALAMENT PERQUE ELS COEFF DE H NO ESTAN ON TOCA'

In [6]:
from qibo.hamiltonians.models import multikron

Id = np.array([[1,0],[0,1]])
Z = np.array([[1,0],[0,-1]])

Z_0 = multikron([Z, Id, Id])
Z_1 = multikron([Id, Z, Id])
Z_2 = multikron([Id, Id, Z])

Z_01 = multikron([Z, Z, Id])
Z_02 = multikron([Z, Id, Z])
Z_12 = multikron([Id, Z, Z])

crafted_H = h_coeffs[0]*Z_0 + h_coeffs[1]*Z_1 + h_coeffs[2]*Z_2 + \
            J_coeffs[0,1]*Z_01 + J_coeffs[0,2]*Z_02 + J_coeffs[1,2]*Z_12

print(h_coeffs)
print(J_coeffs)
print(crafted_H)
print(f'+ offset: {offset}')

for i in range(8):
    print(crafted_H[i,i] + offset)



[-40.618500000000004, -63.986999999999995, -69.6055]
[[ 0.    -3.79   5.065]
 [ 0.     0.     9.88 ]
 [ 0.     0.     0.   ]]
[[-163.056    0.       0.       0.       0.       0.       0.       0.   ]
 [   0.     -53.735    0.       0.       0.       0.       0.       0.   ]
 [   0.       0.     -47.262    0.       0.       0.       0.       0.   ]
 [   0.       0.       0.     101.579    0.       0.       0.       0.   ]
 [   0.       0.       0.       0.     -84.369    0.       0.       0.   ]
 [   0.       0.       0.       0.       0.      45.212    0.       0.   ]
 [   0.       0.       0.       0.       0.       0.      16.265    0.   ]
 [   0.       0.       0.       0.       0.       0.       0.     185.366]]
+ offset: -140.631
-303.687
-194.36599999999999
-187.89300000000003
-39.05200000000001
-225.0
-95.41899999999998
-124.366
44.735000000000014


In [22]:
from Annealing_ham.QKP_ham import QKP_Hamiltonian

qkp_ham = QKP_Hamiltonian(W_capacity, weights, values)

print(qkp_ham.H_target.eigenvalues())

[Qibo 0.2.7|WARNING|2024-05-02 12:22:24]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


[-163.056  -84.369  -53.735  -47.262   16.265   45.212  101.579  185.366]


In [8]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

from DMRG.module import doDMRG_MPO

##### Set bond dimensions and simulation options
chi = 32
Nsites = N

OPTS_numsweeps = 15 # number of DMRG sweeps
OPTS_dispon = 2 # level of output display
OPTS_updateon = True # level of output display
OPTS_maxit = 2 # iterations of Lanczos method
OPTS_krydim = 4 # dimension of Krylov subspace

#####################################

L_dim = M[0].shape[0]
R_dim = M[-1].shape[2]

# Dummy MPO boundary matrices
ML = np.array([1,0,0,0]).reshape(4,1,1) #left MPO boundary
MR = np.array([0,0,0,1]).reshape(4,1,1) #right MPO boundary

# Define the left MPO boundary
ML = np.zeros((L_dim, 1, 1))
ML[0, 0, 0] = 1

MR = np.zeros((R_dim, 1, 1))
MR[-1, 0, 0] = 1

#### Initialize MPS tensors
chid = 2 # local dimension
A = [0 for x in range(Nsites)]
A[0] = np.random.rand(1,chid,min(chi,chid))
for k in range(1,Nsites):
    A[k] = np.random.rand(A[k-1].shape[2],chid,min(min(chi,A[k-1].shape[2]*chid),chid**(Nsites-k-1)))

In [9]:
import random
from collections import defaultdict

class SamplingMPS:

    def _density_matrix(
        self,
        mps: np.array,
        r_site: int,
    ) -> np.array:

        tensor_right = self.tensor_right[r_site + 1]

        return ncon([mps[0], np.conj(mps[0]), tensor_right], [[1, -1, 2], [1, -2, 3], [2, 3]])

    def _obtain_state_i(
        self,
        mps: np.array,
        r_site: int,
    ) -> int:

        reduce_matrix = self._density_matrix(mps=mps, r_site=r_site)

        pb_values = list(np.real(np.diag(reduce_matrix)))

        return (0, pb_values[0]) if random.uniform(0, 1) < pb_values[0] else (1, pb_values[1])

    def _proyecting_state(self, tensor: np.array, state: int) -> np.array:

        state_final = np.array([[1, 0]]) if state == 0 else np.array([[0, 1]])

        return ncon([state_final, tensor], [[-1, 1], [1, -2]])

    def _proyect_state_i(self, mps: np.array, r_site: int) -> np.array:

        if len(mps) > 1:

            mps_new = mps.copy()
            mps_new = mps_new[1:]

        else:
            mps_new = mps.copy()

        state_qubit, pb_state = self._obtain_state_i(mps=mps, r_site=r_site)

        state_to_proyect = mps[0].reshape(2, mps[0].shape[2])

        state_proyected = self._proyecting_state(tensor=state_to_proyect, state=state_qubit)

        state_proyected = state_proyected.reshape(1, mps[0].shape[2])

        mps_new[0] = ncon([state_proyected, mps_new[0]], [[-1, 1], [1, -2, -3]])

        mps_new[0] = 1 / np.sqrt(pb_state) * mps_new[0].reshape(1, 2, mps_new[0].shape[2])

        return mps_new, state_qubit

    def _one_sample(self, mps: np.array) -> np.array:

        mps_sample = mps.copy()

        sample = []

        for i in range(len(mps_sample)):

            mps_sample, state_qubit = self._proyect_state_i(mps=mps_sample, r_site=i)

            sample.append(state_qubit)

        return sample

    def _right_envs(self, mps: np.array):

        ortho_renv = [None for _ in range(len(mps))]

        ortho_renv[-1] = ncon([mps[-1], np.conj(mps[-1])], [[-1, 1, 2], [-2, 1, 2]])

        for s in range(len(mps) - 2, -1, -1):

            ortho_renv[s] = ncon(
                [mps[s], np.conj(mps[s]), ortho_renv[s + 1]],
                [[-1, 1, 2], [-2, 1, 3], [2, 3]],
            )

        ortho_renv.append(np.eye(1))

        return ortho_renv

    def sampling(self, mps: np.array, n_samples: int):

        self.tensor_right = self._right_envs(mps=mps)

        list_samples = [self._one_sample(mps=mps) for _ in range(n_samples)]

        counting = defaultdict(int)

        for sub_list in list_samples:
            key = "".join(map(str, sub_list))
            counting[key] += 1

        return {key: count / n_samples for key, count in counting.items()}

In [10]:
from ncon import ncon

#### Do DMRG sweeps (2-site approach)
En1, left_MPS, sWeight, right_MPS = doDMRG_MPO(A,ML,M,MR,chi,
                                               numsweeps = OPTS_numsweeps,
                                               dispon = OPTS_dispon,
                                               updateon = OPTS_updateon,
                                               maxit = OPTS_maxit,
                                               krydim = OPTS_krydim)

Sweep: 1 of 15, Loc: 1,Energy: -163.043357
Sweep: 1 of 15, Loc: 0,Energy: -163.056000
Sweep: 1 of 15, Loc: 0,Energy: -163.056000
Sweep: 1 of 15, Loc: 1,Energy: -163.056000
Sweep: 2 of 15, Loc: 1,Energy: -163.056000
Sweep: 2 of 15, Loc: 0,Energy: -163.056000
Sweep: 2 of 15, Loc: 0,Energy: -163.056000
Sweep: 2 of 15, Loc: 1,Energy: -163.056000
Sweep: 3 of 15, Loc: 1,Energy: -163.056000
Sweep: 3 of 15, Loc: 0,Energy: -163.056000
Sweep: 3 of 15, Loc: 0,Energy: -163.056000
Sweep: 3 of 15, Loc: 1,Energy: -163.056000
Sweep: 4 of 15, Loc: 1,Energy: -163.056000
Sweep: 4 of 15, Loc: 0,Energy: -163.056000
Sweep: 4 of 15, Loc: 0,Energy: -163.056000
Sweep: 4 of 15, Loc: 1,Energy: -163.056000
Sweep: 5 of 15, Loc: 1,Energy: -163.056000
Sweep: 5 of 15, Loc: 0,Energy: -163.056000
Sweep: 5 of 15, Loc: 0,Energy: -163.056000
Sweep: 5 of 15, Loc: 1,Energy: -163.056000
Sweep: 6 of 15, Loc: 1,Energy: -163.056000
Sweep: 6 of 15, Loc: 0,Energy: -163.056000
Sweep: 6 of 15, Loc: 0,Energy: -163.056000
Sweep: 6 of

In [11]:
# contract the MPS to get the state
E = ncon(right_MPS, [[-1,-2,1],[1,-3,2],[2,-4,-5]])
bulk_E = E.reshape(2,2,2)
print(bulk_E)
print(bulk_E.shape)

for i in range(2):
    for j in range(2):
        for k in range(2):
            print(f'state {i}{j}{k}: {bulk_E[i,j,k]}')

[[[ 1.00000000e+00  9.15741030e-11]
  [ 8.68596469e-09 -4.37875800e-23]]

 [[-2.47576140e-14  1.99135961e-24]
  [ 9.46137821e-09  1.56124397e-09]]]
(2, 2, 2)
state 000: 1.0
state 001: 9.157410301672123e-11
state 010: 8.685964692255931e-09
state 011: -4.378757999714145e-23
state 100: -2.4757614009587254e-14
state 101: 1.9913596133641593e-24
state 110: 9.46137820789472e-09
state 111: 1.5612439694925021e-09


In [12]:
for i in range(len(right_MPS)):
    print(f'MPS[{i}]: ', right_MPS[i].shape)

M[0] = M[0][0, :]
M[-1] = M[-1][:, -1]

for i in range(len(M)):
    print(f'MPO[{i}]: ', M[i].shape)

S_left = ncon(
    [right_MPS[0], M[0], right_MPS[0]], [[1, 2, -1], [-2, 2, 3], [1, 3, -3]]
)
for i in range(1, Nsites - 1):
    tensor = right_MPS[i]
    print(tensor.shape)
    print(M[i].shape)
    S_left = ncon([S_left, tensor, M[i], np.conj(tensor)], [[1, 2, 3], [1, 4, -1], [2, -2, 4, 5], [3, 5, -3]])
S_left = ncon(
    [S_left, right_MPS[-1], M[-1], np.conj(right_MPS[-1])],
    [[1, 2, 3], [1, 4, 5], [2, 4, 6], [3, 6, 5]],
)
print(S_left)

MPS[0]:  (1, 2, 2)
MPS[1]:  (2, 2, 2)
MPS[2]:  (2, 2, 1)
MPO[0]:  (4, 2, 2)
MPO[1]:  (4, 3, 2, 2)
MPO[2]:  (3, 2, 2)
(2, 2, 2)
(4, 3, 2, 2)
-163.05599999999998


In [13]:
'''
VA PERFECTE AQUESTA FUNCIÓ
'''

state = '110'

custom_MPS = []

matrix_0 = np.array([[1, 0]])
reshaped_matrix_0 = matrix_0.reshape(1,2,1)

matrix_1 = np.array([[0, 1]])
reshaped_matrix_1 = matrix_1.reshape(1,2,1)

for s in state:
    if s=='0':
        custom_MPS.append(reshaped_matrix_0)
    elif s=='1':
        custom_MPS.append(reshaped_matrix_1)

S_left = ncon(
    [custom_MPS[0], M[0], custom_MPS[0]], [[1, 2, -1], [-2, 2, 3], [1, 3, -3]]
)
for i in range(1, Nsites - 1):
    tensor = custom_MPS[i]
    print(tensor.shape)
    print(M[i].shape)
    S_left = ncon([S_left, tensor, M[i], np.conj(tensor)], [[1, 2, 3], [1, 4, -1], [2, -2, 4, 5], [3, 5, -3]])
S_left = ncon(
    [S_left, custom_MPS[-1], M[-1], np.conj(custom_MPS[-1])],
    [[1, 2, 3], [1, 4, 5], [2, 4, 6], [3, 6, 5]],
)
print(S_left)

(1, 2, 1)
(4, 3, 2, 2)
16.264999999999993


In [14]:
samplingMPS = SamplingMPS()

result = samplingMPS.sampling(left_MPS, 1000)
print(result)

{'000': 1.0}


In [15]:
Contraer estados que conozca y ver si las energias cuadran

1 0 - estado 0
0 0

0 0 - estado 1
0 1

dev -> modulo de tensor networks

SyntaxError: invalid syntax (4186327484.py, line 1)